<a href="https://colab.research.google.com/github/Kuntya/Udacity/blob/main/LightweightFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRa
* Model: gpt2
* Evaluation approach:
* Fine-tuning dataset:

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [31]:
! pip install -q "datasets==2.15.0"
! pip install peft

In [32]:
# Find a emotion dataset at https://huggingface.co/datasets and load it using the datasets library

from datasets import load_dataset

# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("dair-ai/emotion", split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(2000))

# Show the dataset
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 2000
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 2000
 })}

In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoModelForSequenceClassification

In [34]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
    num_labels=6,
    id2label={0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise" },
    label2id={"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5}
                                            )
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Set the padding token to the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.config.pad_token_id = tokenizer.pad_token_id
# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
      tokenized_dataset[split] = ds[split].map(
        lambda x: tokenizer(x["text"], truncation=True, padding=True),
        batched=True
  )

# Inspect the available columns in the dataset
tokenized_dataset["train"]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 2000
})

In [35]:
print(tokenized_dataset["train"][0])

{'text': 'while cycling in the country', 'label': 4, 'input_ids': [101, 2096, 9670, 1999, 1996, 2406, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [36]:
for split in splits:
    print(f"Tokenized dataset for {split} split:")
    print(tokenized_dataset[split])

    # Check batch sizes and padding
    input_ids_lengths = [len(example["input_ids"]) for example in tokenized_dataset[split]]
    labels_lengths = [len(str(example["label"])) for example in tokenized_dataset[split]]

    print(f"Batch sizes for {split} split:")
    print("Input data:", input_ids_lengths)
    print("Target labels:", labels_lengths)
    print()
    print("Input data:", len(input_ids_lengths))
    print("Target labels:", len(labels_lengths))

Tokenized dataset for train split:
Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 2000
})
Batch sizes for train split:
Input data: [65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65,

In [37]:
#import pandas as pd
#df_pandas = pd.DataFrame(tokenized_dataset["train"])
#df_pandas['idlen']=len(df_pandas['input_ids'])
#df_pandas

In [38]:
for param in model.parameters():
     param.requires_grad = True

print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [39]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/emotions",
        # Set the learning rate
        learning_rate=2e-3,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True),
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,1.613500,1.577401,0.347500


TrainOutput(global_step=2000, training_loss=1.7735562744140625, metrics={'train_runtime': 2915.6733, 'train_samples_per_second': 0.686, 'train_steps_per_second': 0.686, 'total_flos': 33895443564000.0, 'train_loss': 1.7735562744140625, 'epoch': 1.0})

In [40]:
trainer.evaluate()

{'eval_loss': 1.5774006843566895,
 'eval_accuracy': 0.3475,
 'eval_runtime': 357.2425,
 'eval_samples_per_second': 5.598,
 'eval_steps_per_second': 5.598,
 'epoch': 1.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
from peft import LoraConfig, TaskType
config = LoraConfig(r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    fan_in_fan_out=True,
    task_type=TaskType.SEQ_CLS,
    target_modules="classifier")

from peft import get_peft_model
lora_model = get_peft_model(model, config)

In [42]:
lora_model.print_trainable_parameters()

trainable params: 601,398 || all params: 67,565,676 || trainable%: 0.890093958358383


In [43]:
trainer_peft = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/emotions_lora",
        # Set the learning rate
        learning_rate=2e-3,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True),
    compute_metrics=compute_metrics
)

trainer_peft.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.610700,1.578866,0.347500
2,1.627700,1.563357,0.347500
3,1.635500,1.567433,0.347500
4,1.613900,1.564094,0.347500
5,1.562400,1.566106,0.347500


TrainOutput(global_step=5000, training_loss=1.6048740966796875, metrics={'train_runtime': 3090.6827, 'train_samples_per_second': 3.236, 'train_steps_per_second': 1.618, 'total_flos': 172522594316304.0, 'train_loss': 1.6048740966796875, 'epoch': 5.0})

In [51]:
lora_model.save_pretrained("trainer_peft")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [52]:
from peft import AutoPeftModelForCausalLM
lora_model = AutoPeftModelForCausalLM.from_pretrained("trainer_peft")

ValueError: Expected target PEFT class: PeftModelForSequenceClassification, but you have asked for: PeftModelForCausalLM make sure that you are loading the correct model for your task type.

In [48]:
trainer_peft.evaluate()

{'eval_loss': 1.563356876373291,
 'eval_accuracy': 0.3475,
 'eval_runtime': 308.1456,
 'eval_samples_per_second': 6.49,
 'eval_steps_per_second': 3.245,
 'epoch': 5.0}